In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop = nn.Dropout(0.1)
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 16, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
    
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(16)
    
        self.conv4 = nn.Conv2d(16, 8, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(8)

        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(8, 16, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(16)
    
        self.conv6 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(16)

        self.pool3 = nn.MaxPool2d(2, 2)
    
        self.conv7 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn7 = nn.BatchNorm2d(16)

        self.conv8 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn8 = nn.BatchNorm2d(16)
    
        self.conv9 = nn.Conv2d(16, 16, 3, padding=1)
    
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(16,10)


    def forward(self, x):

        x = self.pool1(self.drop(F.relu(self.bn2(self.conv2(self.drop(F.relu(self.bn1(self.conv1(x)))))))))
        x = self.pool2(self.drop(F.relu(self.bn4(self.conv4(self.drop(F.relu(self.bn3(self.conv3(x)))))))))
        x = self.pool3(self.drop(F.relu(self.bn6(self.conv6(self.drop(F.relu(self.bn5(self.conv5(x)))))))))
        x = F.relu(self.bn7(self.conv7(x)))
        x = self.conv9(self.drop(F.relu(self.bn8(self.conv8(x)))))
        x = self.avg_pool(x)
        x = self.fc(x.view(-1, 16))
        return F.log_softmax(x)


In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
           Dropout-3           [-1, 32, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           4,624
       BatchNorm2d-5           [-1, 16, 28, 28]              32
           Dropout-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           2,320
       BatchNorm2d-9           [-1, 16, 14, 14]              32
          Dropout-10           [-1, 16, 14, 14]               0
           Conv2d-11            [-1, 8, 14, 14]           1,160
      BatchNorm2d-12            [-1, 8, 14, 14]              16
    

<ipython-input-2-dfdd41fb9229>:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\n Epoch: {}/20.. Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader, epoch)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-dfdd41fb9229>:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.09756222367286682 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.62it/s]



 Epoch: 1/20.. Test set: Average loss: 0.0727, Accuracy: 9792/10000 (97.92%)



loss=0.0660269483923912 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.81it/s]



 Epoch: 2/20.. Test set: Average loss: 0.0503, Accuracy: 9855/10000 (98.55%)



loss=0.014523806981742382 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.21it/s]


...


 Epoch: 18/20.. Test set: Average loss: 0.0233, Accuracy: 9921/10000 (99.21%)



loss=0.003175369231030345 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.14it/s]



 Epoch: 19/20.. Test set: Average loss: 0.0184, Accuracy: 9939/10000 (99.39%)



loss=0.006978366989642382 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.25it/s]



 Epoch: 20/20.. Test set: Average loss: 0.0216, Accuracy: 9931/10000 (99.31%)

